### Beginning

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
import numpy as np
import h5py, os, glob, re, datetime

In [0]:
hdf_directory = "/content/drive/My Drive/DL/datasets/DIV2K HDF/"
patched_hdf_directory = "/content/drive/My Drive/DL/datasets/DIV2K HDF PATCHED/"

current_time = lambda: datetime.datetime.now().strftime("%m/%d %H:%M:%S")

def get_hdf_dir(training, hsv, patching):
    dataset_type = "train/" if training else "valid/"
    color_model = "HSV/" if hsv else "RGB/" 
    dir_ = patched_hdf_directory if patching else hdf_directory
    return "{}{}{}".format(dir_, color_model, dataset_type)
  
def batch_gen(training, hsv, batch):
    """Create patched minibatches / create nonpatched pairs "low-res"--"hi-res".
        
        
    Parameters
    ----------
        training: bool
            If true, "train" directory is used; 
            if false, "valid" directory is used.
        hsv: bool 
            If true, the HSV color model is used; 
            if false, the RGB color model is used.
        epoch: int
            If even, "bicubic" interpolation is used; 
            if odd, "unknown" interpolation is used (for low-resolution images).
        batch: int
            The minibatch size.
        
        
    Yields
    ------
        I. If "batch" is equal to 1:
            numpy array 
                Low-resolution image.
            numpy array 
                High-resolution image.
            int 
                Image number (counter).
            
        II. If "batch" is greater than 1:
            numpy array 
                Low-resolution image patches.
            numpy array 
                High-resolution image patches.
            int 
                Minibatch number (counter).
         
         
    """
    #if batch == 1, then take nonpatched image
    if batch == 1:
        patching = False
    #else take patched image
    else: patching = True
    hdfs_path = get_hdf_dir(training, hsv, patching)
    hdfs = sorted(glob.glob(hdfs_path + "*.hdf5"))
    if training:
        np.random.shuffle(hdfs)
    batch_counter = 0
    for i in range(len(hdfs)):
        with h5py.File(hdfs[i], "r") as hdf:
            if batch == 1:
                lr = hdf["lr_bicub"][()]
                hr = hdf["hr"][()]
                lr, hr = np.expand_dims(lr, axis=0), np.expand_dims(hr, axis=0)
                yield lr, hr, i
            else:
                range_ = hdf["patched_shape"][()][0]*hdf["patched_shape"][()][1]
                patches_index = list(range(range_))
                np.random.shuffle(patches_index)
                for j in range(len(patches_index) // batch):
                    lr, hr = [], []
                    batch_counter += 1
                    for k in range(batch):
                        lr.append(hdf["lr_bicub"][()][patches_index[j * batch + k]])
                        hr.append(hdf["hr"][()][patches_index[j * batch + k]])
                    np_lr, np_hr = np.asarray(lr), np.asarray(hr)
                    yield np_lr, np_hr, batch_counter       
      
def save_img(img, filename, hsv=False):
    if hsv:
        img = tf.image.hsv_to_rgb(img)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    img_raw = tf.image.encode_png(img).eval()
    return tf.write_file(tf.constant(filename), img_raw)

### Architecture

In [0]:
def conv_layer(x, W_shape, b_shape):
    W = tf.get_variable("W", shape=W_shape, initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable("b", shape=b_shape, initializer=tf.constant_initializer(0.))
    conv = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")
    return tf.nn.bias_add(conv, b)


def edsr_res_block(x, scope):
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        shortcut = x
        with tf.variable_scope("conv1", reuse=tf.AUTO_REUSE) as scope:
            x = conv_layer(x, [5,5,FEATURE_MAPS_NUMBER,FEATURE_MAPS_NUMBER], [FEATURE_MAPS_NUMBER])
        x = tf.nn.relu(x)
        with tf.variable_scope("conv2", reuse=tf.AUTO_REUSE) as scope:
            x = conv_layer(x, [5,5,FEATURE_MAPS_NUMBER,FEATURE_MAPS_NUMBER], [FEATURE_MAPS_NUMBER])
        x = x * 0.1  
    return x + shortcut

# wdsr-a
def wdsr_res_block(x, scope, wide):
    SCALE = 0.1
    
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        shortcut = x
        with tf.variable_scope("conv1", reuse=tf.AUTO_REUSE) as scope:
            x = conv_layer(x, [5,5,FEATURE_MAPS_NUMBER,wide*FEATURE_MAPS_NUMBER], 
                                  [wide*FEATURE_MAPS_NUMBER])
        x = tf.nn.relu(x)
        with tf.variable_scope("conv2", reuse=tf.AUTO_REUSE) as scope:
            x = conv_layer(x, [5,5,wide*FEATURE_MAPS_NUMBER,FEATURE_MAPS_NUMBER], 
                                  [FEATURE_MAPS_NUMBER])
        x = x * SCALE  
    return x + shortcut  

# wdsr-b
def wdsr_res_block_v2(x, scope, wide, kernel):
    LINEAR = 0.8
    SCALE = 0.1
    
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        shortcut = x
        with tf.variable_scope("conv1", reuse=tf.AUTO_REUSE) as scope:
            x = conv_weight_norm(x, kernel, [1, 1, FEATURE_MAPS_NUMBER, wide * FEATURE_MAPS_NUMBER], [wide * FEATURE_MAPS_NUMBER])
        x = tf.nn.relu(x)
        with tf.variable_scope("conv2", reuse=tf.AUTO_REUSE) as scope:
            x = conv_weight_norm(x, kernel, [1, 1, wide * FEATURE_MAPS_NUMBER, int(LINEAR * FEATURE_MAPS_NUMBER)], [int(LINEAR * FEATURE_MAPS_NUMBER)])
        with tf.variable_scope("conv3", reuse=tf.AUTO_REUSE) as scope:
            x = conv_weight_norm(x, kernel, [kernel, kernel, int(LINEAR * FEATURE_MAPS_NUMBER), FEATURE_MAPS_NUMBER], [FEATURE_MAPS_NUMBER])
        x = x * SCALE  
    return x + shortcut 
  
def wdsr_branch(x, kernel, wide, l_fm, scope):
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        with tf.variable_scope("first_conv", reuse=tf.AUTO_REUSE) as scope:
            conv1 = conv_weight_norm(x, kernel, [kernel, kernel, 3, FEATURE_MAPS_NUMBER], [FEATURE_MAPS_NUMBER])   
        #ResBlocks Stack
        res_stack = tf.contrib.layers.repeat(conv1, RES_BLOCKS_NUMBER, wdsr_res_block_v2, 
                                                scope="res_stack", wide=wide, kernel=kernel)
        #Conv before PixelShuffle
        with tf.variable_scope("before_shuffling", reuse=tf.AUTO_REUSE) as scope:
            conv2 = conv_weight_norm(res_stack, kernel, [kernel, kernel, FEATURE_MAPS_NUMBER, l_fm], [l_fm])
        #PixelShuffle
        shuffled = tf.nn.depth_to_space(conv2, UPSCALE_F, data_format='NHWC')
    return shuffled

In [0]:
def edsr(x, UPSCALE_F):
    #First Conv
    with tf.variable_scope("first_conv", reuse=tf.AUTO_REUSE) as scope:
        conv1 = conv_layer(x, [5,5,3,FEATURE_MAPS_NUMBER], [FEATURE_MAPS_NUMBER])    
    shortcut1 = conv1
    #ResBlocks Stack
    res_stack = tf.contrib.layers.repeat(conv1, RES_BLOCKS_NUMBER, edsr_res_block, 
                                            scope="res_stack")
    #Conv after ResBlocks
    with tf.variable_scope("after_res_block", reuse=tf.AUTO_REUSE) as scope:
        conv2 = conv_layer(res_stack, [5,5,FEATURE_MAPS_NUMBER,FEATURE_MAPS_NUMBER], 
                                          [FEATURE_MAPS_NUMBER])
    #Shortcut
    res1 = 0.1 * conv2 + shortcut1
    #Conv before PixelShuffle
    with tf.variable_scope("before_shuffling", reuse=tf.AUTO_REUSE) as scope:
        conv3 = conv_layer(res1, [5,5,FEATURE_MAPS_NUMBER,FEATURE_MAPS_NUMBER*UPSCALE_F], 
                                      [FEATURE_MAPS_NUMBER*2])
    #PixelShuffle
    shuffled = tf.nn.depth_to_space(conv3, UPSCALE_F, data_format='NHWC')
    #Conv to 3 channels
    with tf.variable_scope("last_conv", reuse=tf.AUTO_REUSE) as scope:
        conv4 = conv_layer(shuffled, [5,5,FEATURE_MAPS_NUMBER/UPSCALE_F,3], [3])
    return conv4

def wdsr(x, UPSCALE_F):
    N_FM_LAST = 3 * UPSCALE_F ** 2
    WIDE = 6
    
    #Skip Branch
    with tf.variable_scope("0_before_shuffling", reuse=tf.AUTO_REUSE) as scope:
        conv0 = conv_weight_norm(x, 5, [5, 5, 3, N_FM_LAST], [N_FM_LAST])
    shuffled_1 = tf.nn.depth_to_space(conv0, UPSCALE_F, data_format='NHWC')
  
    #Main Branch
    shuffled_2 = wdsr_branch(x, 5, WIDE, N_FM_LAST, "main_branch")

    #Sum
    out = shuffled_1 + shuffled_2
    return out

In [0]:
UPSCALE_F = 2
FEATURE_MAPS_NUMBER = 64
RES_BLOCKS_NUMBER = 1

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, None, None, 3], "LR_image")
y = tf.placeholder(tf.float32, [None, None, None, 3], "HR_image")
y_net = edsr(x, UPSCALE_F)

### Loss

In [0]:
l2_lambda = tf.placeholder(tf.float32, name="L2_lambda")
vars_ = [var for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)]
regularization = tf.multiply(tf.reduce_sum([tf.nn.l2_loss(var) for var in vars_]), l2_lambda)

learning_rate = tf.placeholder(tf.float32, name="Learning_rate")
mae = tf.reduce_mean(tf.abs(tf.subtract(y_net, y)))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(tf.add(mae, regularization))

### Training

In [0]:
TRAINING = True
HSV = True
LAST_EPOCH = 3
LEARNING_RATE = 5e-4
LAMBDA = 1e-3
BATCHSIZE = 16

print("Choose your initial epoch:")
INITIAL_EPOCH = int(input())
print("That's a great choice!")

if HSV: 
    save_path = "/content/drive/My Drive/DL/ckpt/SISR HSV vs RGB/edsr/SR_HSV_edsr_"
    log_path = "/content/drive/My Drive/DL/metrics/SISR HSV vs RGB/loss_hsv_edsr.txt"
else:    
    save_path = "/content/drive/My Drive/DL/ckpt/SISR HSV vs RGB/edsr/SR_RGB_edsr_"
    log_path = "/content/drive/My Drive/DL/metrics/SISR HSV vs RGB/loss_rgb_edsr.txt"
    
    
print("Start:", current_time())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=1001)
    if INITIAL_EPOCH == 0:
        with open(log_path, "w") as txt_file: pass
        print("OK, it's a new training (without disconnect, please BlessRNG).\n")
    else: 
        saver.restore(sess, "{}{}.ckpt".format(save_path, INITIAL_EPOCH))
        print("Сontinue training from a {} epoch (without disconnect, please BlessRNG).\n".format(INITIAL_EPOCH))
    for epoch in range(INITIAL_EPOCH, LAST_EPOCH):
        #amazing robot-face condition
        if epoch != 0 == 0 != epoch:
            LEARNING_RATE = LEARNING_RATE / (4.5 * epoch)
        print("A new epoch is begining ({})... LR = {}".format(epoch + 1, LEARNING_RATE))
        for img_x, img_y, num in batch_gen(TRAINING, HSV, BATCHSIZE):
            _, loss = sess.run([ train_step, mae ], feed_dict={ x: img_x, y: img_y, learning_rate: LEARNING_RATE, l2_lambda: LAMBDA })
            with open(log_path, "a+") as txt_file:
                txt_file.write("time: {}... epoch {}... batch {}... loss = {:.7f}\n".format(current_time(), epoch + 1, num, loss))
            if num % 2000 == 0:
                saver.save(sess, "{}autosave_{}.ckpt".format(save_path, num))
            if num % 300 == 0:
                print("{}... mb {} ({})... loss = {:.7f}".format(current_time(), num, epoch + 1, loss))
        saver.save(sess, "{}{}.ckpt".format(save_path, epoch + 1))      
    
print("\nEnd:", current_time())    

### Result

In [0]:
TRAINING = False
HSV = False
UPSCALE_F = 2
BATCHSIZE = 1
print("Choose your restoring epoch:")
EPOCH = int(input())
test_images = [8, 40, 52]

if HSV: 
    ckpt_path = "/content/drive/My Drive/DL/ckpt/SISR HSV vs RGB/edsr/SR_HSV_edsr_"
    log_path = "/content/drive/My Drive/DL/metrics/SISR HSV vs RGB/psnr_ssim_hsv_esdr.txt"
    img_save_path = "/content/drive/My Drive/DL/test_img/edsr/HSV/"
else:  
    ckpt_path = "/content/drive/My Drive/DL/ckpt/SISR HSV vs RGB/edsr/SR_RGB_edsr_"
    log_path = "/content/drive/My Drive/DL/metrics/SISR HSV vs RGB/psnr_ssim_rgb_edsr.txt"
    img_save_path = "/content/drive/My Drive/DL/test_img/edsr/RGB/"


    
print("Start:", current_time())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, "{}{}.ckpt".format(ckpt_path, EPOCH))
    get_upsc_img = edsr(x, UPSCALE_F)
    if EPOCH == 1:
            with open(log_path, "w") as txt_file: pass
    for lr_img, orig_img, num in batch_gen(TRAINING, HSV, BATCHSIZE):
        if num in test_images:
            upsc_img = sess.run(get_upsc_img, feed_dict={ x : lr_img })
            upsc_img[upsc_img > 1] = 1
            upsc_img[upsc_img < 0] = 0
            upsc_img_tensor, orig_img_tensor = tf.convert_to_tensor(upsc_img), tf.convert_to_tensor(orig_img)
            ssim, psnr = tf.image.ssim(upsc_img_tensor, orig_img_tensor, max_val=1.0).eval(), tf.image.psnr(upsc_img_tensor, orig_img_tensor, max_val=1.0).eval()
            img_saving = save_img(upsc_img[0], "{}{}_epoch{}.png".format(img_save_path, num + 1, EPOCH), hsv=HSV)
            sess.run(img_saving)
            with open(log_path, "a+") as txt_file:
                txt_file.write("time: {}... epoch {}... image {}... psnr {:.20f}... ssim {:.20f}\n".format(current_time(), EPOCH, num + 1, psnr[0], ssim[0]))
            print("time: {}... epoch {}... image {}... psnr {:.7f}... ssim {:.7f}".format(current_time(), EPOCH, num + 1, psnr[0], ssim[0]))
            if num == test_images[-1]: break
    
print("End:", current_time())

### Inference

In [0]:
def path2image_inference(img_path):
    img_raw = tf.read_file(img_path)
    img_dec = tf.image.decode_image(img_raw, channels=3)
    img = tf.image.convert_image_dtype(img_dec, tf.float32).eval()
    return img
  
def save_img_inference(img, filename):
    img = tf.image.convert_image_dtype(img, tf.uint8)
    img_raw = tf.image.encode_png(img).eval()
    return tf.write_file(tf.constant(filename), img_raw)

with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, "/content/drive/My Drive/DL/ckpt/SISR HSV vs RGB/edsr/SR_RGB_edsr3.ckpt")
    get_upsc_img = edsr(x, UPSCALE_F)
    for img_path in glob.glob("/content/drive/My Drive/DL/test_img/edsr/_inference/*"):
        img = path2image_inference(img_path)
        img = np.expand_dims(img, axis=0)
        upsc_img = sess.run(get_upsc_img, feed_dict={ x : img })
        upsc_img[upsc_img > 1] = 1
        upsc_img[upsc_img < 0] = 0
        new_img_path = "{}__x2.png".format(os.path.splitext(re.split(r"/", img_path)[-1])[0])
        img_saving = save_img_inference(upsc_img[0], "/content/drive/My Drive/DL/test_img/edsr/_inference/{}".format(new_img_path))
        sess.run(img_saving)
        print("Image {} done.".format(new_img_path))
        

### Bicubic interpolation PSNR / SSIM

In [0]:
HSV = False
img_num = ["0809", "0841", "0853"]
hr_path = lambda img_num: "/content/drive/My Drive/DL/test_img/edsr/bicub/{}.png".format(img_num)
bicub_path = lambda img_num: "/content/drive/My Drive/DL/test_img/edsr/bicub/{}_bicub.png".format(img_num)

def path2image(img_path, hsv):
    img_raw = tf.read_file(img_path)
    img_dec = tf.image.decode_png(img_raw, channels=3)
    img = tf.image.convert_image_dtype(img_dec, tf.float32).eval()
    if hsv: img = tf.image.rgb_to_hsv(img).eval()
    return img
  
tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    for i in range(len(img_num)):
        hr, bicub = path2image(hr_path(img_num[i]), HSV), path2image(bicub_path(img_num[i]), HSV)
        hr_tensor, bicub_tensor = tf.convert_to_tensor(hr), tf.convert_to_tensor(bicub)
        ssim, psnr = tf.image.ssim(hr_tensor, bicub_tensor, max_val=1.0).eval(), tf.image.psnr(hr_tensor, bicub_tensor, max_val=1.0).eval()
        print("{}: PSNR {}... SSIM {}".format(img_num[i], psnr, ssim))